In [ ]:
import requests
import bs4 as beautifulsoap
import pandas as pd

In [ ]:
# The original url
url = "http://lhr.nu.edu.pk/faculty/"
# making a request for geting the url
request = requests.get(url)
try:
  request.raise_for_status()
  # checking if the url gives successful http codes
  print("Working on it!")
  print("Successfully opened the URL!")
# requesting exception in the form of m and then displaying it as an error
except requests.exceptions.RequestException as m:
  print(f"Error while opening url! {m}")



Working on it!
Successfully opened the URL!


In [ ]:
# parsing the html contents of the main page
soup = beautifulsoap.BeautifulSoup(request.text, "html.parser")
# finding the links to all departments
departments = soup.find_all(class_ =  "mt-5 btn btn-nu white")
# finding the department names from the above links
department_names = [dep.text for dep in departments]
# extracting the url for each department page
department_url= [dep.get("href") for dep in departments]

In [ ]:
  # first of all finding the FSC though id
def all_Information(request, myID):
    fsc = soup.find(id= myID)
    # getting the name of the emplyees in FSC through text centre
    fsc_employee_id = fsc.find_all("h5","text-center")
    # makes a list of names and puts in text from fsc_emplyee_id
    name = [i.text for i in fsc_employee_id]

    # now finding the tag that contains designation
    desig = fsc.find_all('p', class_="small text-center font-italic")
    # making a list of all the designations and slpits them from where they ontain a break
    designation = [designation.text.strip().split('\n') for designation in desig]
    temp = []
    # simply just appending in a empty list
    for i in designation:
      temp.append(i[0])

    #  Now checking if faculy is approved by hec or not
    approval = [designation.find("br") != -1 for designation in desig]
    # appending true in temp1 if the loop comes to a br
    temp1= []
    for i in range(len(desig)):
      if(desig[i].find('br')):
        temp1.append(True)
      else:
        temp1.append(False)

    #  now finding the email through the tags and the class
    email = fsc.find_all('p', class_ = "mb-0 text-center")
    #  simply going through all the emails and placing them in a list
    email = [i.text for i in email]

    #  now we find the department through h1 tag
    dept = fsc.find('h1')

    # finding the images through the tags and then placing them in a list
    my_img = fsc.find_all("img", class_ = "card-img-top rounded-circle mt-3 mb-0 d-block mx-auto")
    img = [img['src'] for img in my_img]

    my_id = fsc.find_all("a","faculty-link")# Geting the ID of Facuty Member
    ids = [link.get("href").split('/')[3] for link in my_id]
     # just organizing as a dataframe with id giving as range of total no of employees
    whole_data = pd.DataFrame({'ID' : ids,
                                            "Name " : name,
                                            'Designation': temp,
                                            "HEC Approval": temp1,
                                            "Department": dept,
                                            "Email": email,
                                            "Image": img})
    #  turning the type to .csv according to the departement
    whole_data.to_csv(myID + '.csv', index = False)
    return whole_data
listt = ["fsc", "ee", "cv", "ss", "fsm"]
for i in listt:
    all_Information(request, i)


In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

def some_Information(request, myID):
    # First loading the file as CSV
    info = pd.read_csv(myID + '.csv')
    # Getting the IDs of the faculty members
    ids = info['ID']
    # Creating a list of all the URLs of the faculty members
    URLs = [f"https://lhr.nu.edu.pk/{myID}/facultyProfile/" + str(i) for i in ids]
    # Creating temp and empty lists
    highestEducation = []
    extension = []
    # Looping through the URLS
    for url in URLs:
        # Getting the URL
        req = requests.get(url)
        # Getting the contents of the page of URL
        soup1 = BeautifulSoup(req.content, "html.parser")
        # Finding the education of the faculty members
        education = soup1.find("div", class_="col-lg-8 col-md-6 col-sm-12 text-justify")
        if education:
            edu_items = education.find_all('li')  # Get all list items
            highestEducation.append([item.text for item in edu_items])  # Collect all items
        else:
            highestEducation.append(None)
        # Finding extension
        ext = soup1.find("span", class_="small")
        if ext:
           extension_text = ext.text.split(":")[1].strip() if ":" in ext.text else None
           extension.append(extension_text)
        else:
            extension.append(None)
    # Creating the DataFrame
    whole_Data = pd.DataFrame({'ID': ids, "Highest Education": highestEducation, "Extension": extension})
    # Saving to CSV according to the department
    whole_Data.to_csv(myID + '2.csv', index=False)
    return whole_Data
# Call the function with appropriate parameters
ids = ["fsc", "ee", "cv", "ss", "fsm"]
for id in ids:
  some_Information(request, id)


In [ ]:
# creating temp for storing all the data
some_data = {}
some_more_data = {}
def some_More_Information(request, myID):
  # reading from the first csv file and then storing in temp
  data = pd.read_csv(myID + ".csv")
  some_data[myID] = data
  # reading from the second csv file and then stroing in temp2
  data2 = pd.read_csv(myID + "2.csv")
  some_more_data[myID] = data2
  # merging data based on id column and with left join
  some_data[myID] = pd.merge(some_data[myID], some_more_data[myID], on = 'ID', how = 'left')

  # Simple concatination while ignoring indexes
  THIS_IS_WHOLE_DATA = pd.concat(some_data.values(), ignore_index=True)
  # converting to a new csv file
  THIS_IS_WHOLE_DATA.to_csv("fast_lahore_faculty.csv", index = False)
  return THIS_IS_WHOLE_DATA
ids = ["fsc", "ee","cv", "ss", "fsm"]
for id in ids:
  some_More_Information(request, id)